In [6]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
data = pd.read_csv('data.csv', encoding='UTF-8')

In [7]:
representativity = data['CAUSAL_PARALIZACION'].value_counts(normalize=True)
top = representativity.head(3).index
data['CAUSAL_PARALIZACION'] = data['CAUSAL_PARALIZACION'].apply(
    lambda x: x if x in top else '14 Otros'
)
columns_categorical = data.select_dtypes(include=['object']).columns
for col in columns_categorical:
    data[col] = data[col].str.upper().str.strip()

In [8]:
cols_normalize_currency = ['COSTO_DE_OBRA-EXP_TEC',
                           'COSTO_ACTUALIZADO',
                           'SALDO_DE_INVERSION']
for col in cols_normalize_currency:
    data[col] = data[col].str.replace(',', '', regex=True)
    data[col] = data[col].replace('', None)
    data[col] = data[col].dropna().str.replace(
        r'[^\d.]',
        '',
        regex=True).str.replace(
        r'^\.',
        '',
        regex=True).astype(float, errors='ignore')
    data[col] = pd.to_numeric(data[col], errors='coerce')

In [9]:
val_range = data['RANGO_DE_INVERSION'].unique()
data_sets = []
for r in val_range:
    data_sets.append(data[data['RANGO_DE_INVERSION'] == r])

In [10]:
for index in range(len(val_range)):
    cols_numerical = data_sets[index].select_dtypes(
        include=['float64', 'int64']).columns
    cols_categorical = data_sets[index].select_dtypes(
        include=['object']).columns
    numerical_imputer = IterativeImputer(random_state=42)
    data_sets[index].loc[:, cols_numerical] = numerical_imputer.fit_transform(
        data_sets[index][cols_numerical])

    for col in cols_categorical:
        data_sets[index].loc[:, col] = data_sets[index][col].fillna(
            data_sets[index][col].mode()[0])
data_cleaning = pd.concat(data_sets, ignore_index=True)
data_cleaning['ANO_INICIO_OBRA'] = data_cleaning['ANO_INICIO_OBRA'].astype(int)
data_cleaning['PLAZO_DE_EJECUCION'] = data_cleaning['PLAZO_DE_EJECUCION'].astype(int)
data_cleaning['ULT_AVANCE_MES'] = data_cleaning['ULT_AVANCE_MES'].astype(int)
data_cleaning['ULT_AVANCE_ANO'] = data_cleaning['ULT_AVANCE_ANO'].astype(int)
data_cleaning.to_csv('dataset-cleaning.csv', encoding='utf-8', index=False)